# Modelos de Pronóstico para Negocios

En este notebook aprenderás a aplicar modelos de pronóstico (forecasting) utilizando bases de datos educativas. Exploraremos desde los conceptos básicos hasta la implementación de algoritmos clásicos de series de tiempo, con ejemplos prácticos y visualizaciones.

---

## 1. Importar librerías y clonar repositorio de bases de datos

Comenzaremos importando las librerías necesarias y clonando el repositorio educativo que contiene bases de datos en formato SQLite.

In [1]:
# Importar librerías principales
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, adfuller
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Clonar el repositorio de bases de datos educativas (solo la primera vez)
!git clone https://github.com/davidjamesknight/SQLite_databases_for_learning_data_science.git

# Cambiar al directorio del repositorio
%cd SQLite_databases_for_learning_data_science

fatal: destination path 'SQLite_databases_for_learning_data_science' already exists and is not an empty directory.
/Users/jorgeramirez/Documents/Python Projects/2 Pronósticos/SQLite_databases_for_learning_data_science


## 2. Explorar tablas y buscar series de tiempo

Nos conectaremos a la base de datos y listaremos las tablas disponibles. Buscaremos alguna tabla que contenga datos de series de tiempo (por ejemplo, columnas de fecha y valores numéricos).

In [2]:
# Conectarse a la base de datos flights.db
db = sqlite3.connect('flights.db')
cursor = db.cursor()

# Listar todas las tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablas = cursor.fetchall()
print("Tablas encontradas:")
for tabla in tablas:
    print("-", tabla[0])

# Revisar las primeras filas de cada tabla para buscar series de tiempo
for tabla in tablas:
    nombre_tabla = tabla[0]
    print(f"\nPrimeras filas de la tabla '{nombre_tabla}':")
    df_temp = pd.read_sql_query(f"SELECT * FROM {nombre_tabla} LIMIT 5", db)
    display(df_temp)

Tablas encontradas:
- Observation
- Month

Primeras filas de la tabla 'Observation':


,year,passengers,month_id
0,1949,112,0
1,1949,118,1
2,1949,132,2
3,1949,129,3
4,1949,121,4



Primeras filas de la tabla 'Month':


,month_id,month
0,0,January
1,1,February
2,2,March
3,3,April
4,4,May


## 3. Antecedentes y generalidades de los algoritmos para forecasting

El **pronóstico** (forecasting) es el proceso de estimar valores futuros basados en datos históricos. Es fundamental en negocios para anticipar ventas, demanda, inventarios, precios, entre otros.

**Aplicaciones típicas:**
- Predicción de ventas mensuales o anuales.
- Estimación de demanda de productos.
- Planeación de recursos y presupuestos.
- Proyecciones financieras.

Los algoritmos de forecasting pueden ser simples (promedios móviles) o avanzados (modelos estadísticos y de machine learning).

## 4. Objetivos del desarrollo de pronósticos en la toma de decisiones

Los pronósticos permiten a las empresas:
- Optimizar inventarios y reducir costos de almacenamiento.
- Mejorar la planeación financiera y presupuestal.
- Anticipar cambios en la demanda y ajustar estrategias de marketing.
- Tomar decisiones informadas sobre producción, compras y recursos humanos.

Un buen pronóstico ayuda a minimizar riesgos y aprovechar oportunidades en mercados dinámicos.

## 5. Identificar y preparar una serie de tiempo

Seleccionaremos una tabla adecuada, extraeremos una columna de fecha y una de valores, y prepararemos los datos para análisis de series de tiempo.

In [3]:
# Query explícito: unir Observation, Month y Airport para obtener una serie de tiempo de pasajeros mensuales

sql = """
SELECT
    O.year,
    M.month,
    O.passengers
FROM
    Observation AS O
JOIN
    Month AS M ON O.month_id = M.month_id
ORDER BY
    O.year, M.month_id
"""

df = pd.read_sql_query(sql, db)

# Crear columna de fecha
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'], format='%Y-%B')
df = df.sort_values('date')
df.set_index(
    'date',
    inplace=True
)
df_ts = df['passengers']
df.head()

,year,month,passengers
date,,,
1949-01-01,1949,January,112
1949-02-01,1949,February,118
1949-03-01,1949,March,132
1949-04-01,1949,April,129
1949-05-01,1949,May,121


## 6. Visualización de la serie de tiempo

Graficaremos la serie de tiempo para observar tendencias, estacionalidad y posibles anomalías.

In [4]:
fig = px.line(
    df_ts,
    title='Pasajeros mensuales de vuelos'
)

fig.update_layout(
    xaxis_title='Fecha',
    yaxis_title='Número de pasajeros'
)

fig.show()

## 7. Algoritmos de forecasting para series de tiempo

Los principales algoritmos de forecasting se clasifican en:

- **Modelos clásicos:** Promedios móviles, suavización exponencial, ARIMA, SARIMA, Holt-Winters.
- **Modelos de machine learning:** Regresión, Random Forest, XGBoost, redes neuronales.
- **Modelos avanzados:** Prophet (Facebook), modelos de descomposición, modelos híbridos.

En este notebook nos enfocaremos en los modelos clásicos, ideales para datos de series de tiempo univariadas.

## 8. Métodos de promedios móviles

Los promedios móviles son una técnica simple pero efectiva para suavizar las fluctuaciones de corto plazo en una serie de tiempo y resaltar tendencias a largo plazo. Se basan en calcular el promedio de un número fijo de puntos de datos recientes.

- **Promedio móvil simple (SMA):** Calcula el promedio aritmético de los últimos *n* valores de la serie de tiempo. Cada punto de datos en la ventana de tiempo tiene el mismo peso.

  **Fundamento matemático:**
  El SMA para un período *n* en el tiempo *t* se calcula como:
  $$SMA_t = \frac{P_t + P_{t-1} + ... + P_{t-n+1}}{n}$$
  Donde:
  - $P_t$ es el valor de la serie de tiempo en el tiempo *t*.
  - *n* es el número de períodos en la ventana del promedio móvil.

  **Fundamento estadístico:**
  El SMA es una estimación de la media local de la serie de tiempo. Reduce el ruido aleatorio y suaviza la serie, haciendo que las tendencias sean más visibles. Sin embargo, puede rezagarse detrás de los cambios bruscos en la tendencia.

- **Promedio móvil ponderado (WMA):** Asigna diferentes pesos a los valores dentro de la ventana de tiempo, generalmente dando más peso a los datos más recientes. Esto permite que el WMA reaccione más rápidamente a los cambios en la serie que el SMA.

  **Fundamento matemático:**
  El WMA para un período *n* en el tiempo *t* se calcula como:
  $$WMA_t = \frac{w_n P_t + w_{n-1} P_{t-1} + ... + w_1 P_{t-n+1}}{w_n + w_{n-1} + ... + w_1}$$
  Donde:
  - $P_t$ es el valor de la serie de tiempo en el tiempo *t*.
  - $w_i$ es el peso asignado al valor en el tiempo $t-i+1$. Los pesos suelen disminuir linealmente o exponencialmente a medida que los datos se vuelven más antiguos.

  **Fundamento estadístico:**
  El WMA es también una estimación de la media local, pero con un énfasis en los datos recientes. La elección de los pesos es crucial y depende de la naturaleza de la serie y del objetivo del análisis. Un WMA con pesos decrecientes linealmente es un caso común.

A continuación, implementamos el promedio móvil simple.

In [5]:
df['SMA_12'] = df_ts.rolling(window=12).mean()
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_ts.index,
        y=df_ts,
        mode='lines',
        name='Pasajeros reales',
        opacity=0.5
    )
)

fig.add_trace(
    go.Scatter(
        x=df['SMA_12'].index,
        y=df['SMA_12'],
        mode='lines',
        name='Promedio móvil 12 meses'
    )
)

fig.update_layout(
    title='Promedio móvil de 12 meses',
    xaxis_title='Fecha',
    yaxis_title='Número de pasajeros'
)

fig.show()

## 9. Métodos de suavización exponencial

La **suavización exponencial** es una técnica de pronóstico que asigna pesos que disminuyen exponencialmente a medida que los datos se vuelven más antiguos. Esto permite que los modelos se adapten mejor a cambios recientes en la serie de tiempo.


Estos métodos son ampliamente usados por su simplicidad y eficacia.


Existen varios métodos de suavización exponencial, adecuados para diferentes tipos de series:

### **9.1. Suavización exponencial simple (SES):** Para series sin tendencia ni estacionalidad.
  
  **Fundamento matemático:**
  La fórmula de recurrencia para SES es:
  $$F_{t+1} = \alpha Y_t + (1 - \alpha) F_t$$
  Donde:
  - $F_{t+1}$ es el pronóstico para el período $t+1$.
  - $Y_t$ es el valor real de la serie en el período $t$.
  - $F_t$ es el pronóstico para el período $t$.
  - $\alpha$ es el parámetro de suavización (0 < $\alpha$ < 1).
  
  **Fundamento estadístico:**
  SES es equivalente a un modelo ARIMA(0,1,1) sin constante. Asume que el nivel de la serie cambia aleatoriamente con el tiempo y que los errores de pronóstico son ruido blanco. Un valor de $\alpha$ cercano a 1 da más peso a las observaciones recientes, mientras que un valor cercano a 0 da más peso al historial pasado.

Aplicamos el método de suavización exponencial simple usando `statsmodels`. Es ideal para series sin tendencia ni estacionalidad marcada.

In [6]:
ses_model = SimpleExpSmoothing(df_ts).fit(smoothing_level=0.2, optimized=False)
df['SES'] = ses_model.fittedvalues

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df_ts.index,
        y=df_ts,
        mode='lines',
        name='Pasajeros reales'
    )
)

fig.add_trace(
    go.Scatter(
        x=df['SES'].index,
        y=df['SES'],
        mode='lines',
        name='Suavización exponencial simple'
    )
)

fig.update_layout(
    title='Suavización exponencial simple',
    xaxis_title='Fecha',
    yaxis_title='Número de pasajeros'
)

fig.show()

### **9.2. Doble suavización exponencial (Método de Brown):** Para series con tendencia lineal pero sin estacionalidad. Utiliza dos ecuaciones de suavización para estimar el nivel y la tendencia.

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Brown son:
  Nivel suavizado simple: $$S_t = \alpha Y_t + (1 - \alpha) S_{t-1}$$
  Nivel doblemente suavizado: $$S'_t = \alpha S_t + (1 - \alpha) S'_{t-1}$$
  La tendencia se estima como: $$b_t = \frac{\alpha}{1-\alpha} (S_t - S'_t)$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = a_t + h b_t$$
  Donde:
  - $S_t$ es el nivel suavizado simple en el tiempo *t*.
  - $S'_t$ es el nivel doblemente suavizado en el tiempo *t*.
  - $a_t = S_t + b_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $\alpha$ es el parámetro de suavización (0 < $\alpha$ < 1).

  **Fundamento estadístico:**
  El método de Brown es un caso especial del método de Holt. Asume que la serie tiene una tendencia lineal aditiva y que el nivel y la pendiente cambian aleatoriamente. Es equivalente a un modelo ARIMA(0,2,2) sin constante.

In [7]:
brown_model = ExponentialSmoothing(df_ts, trend='add', seasonal=None).fit()
df['Brown'] = brown_model.fittedvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ts.index, y=df_ts, mode='lines', name='Pasajeros reales'))
fig.add_trace(go.Scatter(x=df['Brown'].index, y=df['Brown'], mode='lines', name='Método de Brown'))
fig.update_layout(title='Doble suavización exponencial (Brown)', xaxis_title='Fecha', yaxis_title='Número de pasajeros')
fig.show()


### **9.3. Método de Holt:** Extiende la suavización exponencial simple para manejar series con tendencia lineal (aditiva o multiplicativa), pero sin estacionalidad. Utiliza dos parámetros de suavización: uno para el nivel ($\alpha$) y otro para la tendencia ($\beta$).

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Holt (con tendencia aditiva) son:
  Nivel: $$L_t = \alpha Y_t + (1 - \alpha) (L_{t-1} + b_{t-1})$$
  Tendencia: $$b_t = \beta (L_t - L_{t-1}) + (1 - \beta) b_{t-1}$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = L_t + h b_t$$
  Donde:
  - $L_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $\alpha$ es el parámetro de suavización del nivel (0 < $\alpha$ < 1).
  - $\beta$ es el parámetro de suavización de la tendencia (0 < $\beta$ < 1).

  **Fundamento estadístico:**
  El método de Holt es equivalente a un modelo ARIMA(0,2,2) cuando se utiliza tendencia aditiva. Permite que el nivel y la tendencia se ajusten a los cambios en la serie.

In [8]:
holt_model = Holt(df_ts).fit()
df['Holt'] = holt_model.fittedvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ts.index, y=df_ts, mode='lines', name='Pasajeros reales'))
fig.add_trace(go.Scatter(x=df['Holt'].index, y=df['Holt'], mode='lines', name='Método de Holt'))
fig.update_layout(title='Suavización exponencial de Holt', xaxis_title='Fecha', yaxis_title='Número de pasajeros')
fig.show()

### **9.4. Método de Holt-Winters:** Permite modelar series con tendencia y estacionalidad (aditiva o multiplicativa). Agrega un tercer parámetro de suavización ($\gamma$) para el componente estacional.

  **Fundamento matemático:**
  Las fórmulas de recurrencia para el método de Holt-Winters (con tendencia y estacionalidad aditivas) son:
  Nivel: $$L_t = \alpha (Y_t - S_{t-p}) + (1 - \alpha) (L_{t-1} + b_{t-1})$$
  Tendencia: $$b_t = \beta (L_t - L_{t-1}) + (1 - \beta) b_{t-1}$$
  Estacionalidad: $$S_t = \gamma (Y_t - L_t) + (1 - \gamma) S_{t-p}$$
  El pronóstico para *h* períodos futuros es: $$F_{t+h} = L_t + h b_t + S_{t+h-p}$$
  Donde:
  - $L_t$ es el nivel estimado en el tiempo *t*.
  - $b_t$ es la tendencia estimada en el tiempo *t*.
  - $S_t$ es el componente estacional estimado en el tiempo *t*.
  - $\alpha$, $\beta$, $\gamma$ son los parámetros de suavización (0 < $\alpha$, $\beta$, $\gamma$ < 1).
  - *p* es el período estacional (por ejemplo, 12 para datos mensuales con estacionalidad anual).

  **Fundamento estadístico:**
  El método de Holt-Winters (aditivo) es equivalente a un modelo SARIMA(0,1,p)(0,1,0)p. Modela la serie como la suma de un nivel, una tendencia y un componente estacional periódico, más un error aleatorio. Permite que el nivel, la tendencia y la estacionalidad se adapten a los cambios en la serie. La elección entre modelos aditivos y multiplicativos depende de si la magnitud de la estacionalidad es independiente o proporcional al nivel de la serie.

In [9]:
hw_model = ExponentialSmoothing(df_ts, trend='add', seasonal='add', seasonal_periods=12).fit()
df['HW'] = hw_model.fittedvalues
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ts.index, y=df_ts, mode='lines', name='Pasajeros reales'))
fig.add_trace(go.Scatter(x=df['HW'].index, y=df['HW'], mode='lines', name='Holt-Winters'))
fig.update_layout(title='Suavización exponencial Holt-Winters', xaxis_title='Fecha', yaxis_title='Número de pasajeros')
fig.show()

## 10. Descomposición de series de tiempo

La descomposición permite separar la serie en componentes de tendencia, estacionalidad y residuales. Esto ayuda a entender mejor el comportamiento de la serie.

In [10]:
result = seasonal_decompose(df_ts, model='additive', period=12)
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=['Serie original', 'Tendencia', 'Estacionalidad', 'Residuo'])
fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, name='Original'), row=1, col=1)
fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, name='Tendencia'), row=2, col=1)
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, name='Estacionalidad'), row=3, col=1)
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, name='Residuo'), row=4, col=1)
fig.update_layout(height=900, title_text='Descomposición de la serie de tiempo')
fig.show()

## 11. Aplicación del algoritmo en plataformas de ML y software especializado

Además de Python y statsmodels, existen otras plataformas para forecasting:

- **scikit-learn:** Permite usar regresión y modelos de ML para pronóstico.
- **Azure ML, Power BI, Google Cloud AI:** Plataformas empresariales con módulos de forecasting.
- **Prophet (Facebook):** Herramienta especializada para series de tiempo con estacionalidad compleja.

A continuación, un ejemplo de ajuste y predicción usando statsmodels:

In [11]:
# Pronóstico futuro con Holt-Winters y comparación de errores de los métodos

from sklearn.metrics import mean_squared_error

# Pronóstico futuro con Holt-Winters (14 meses)
n_forecast = 14
forecast = hw_model.forecast(n_forecast)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['passengers'], mode='lines', name='Pasajeros reales'))
fig.add_trace(go.Scatter(x=df.index, y=df['HW'], mode='lines', name='Holt-Winters (ajuste)'))
future_dates = pd.date_range(df.index[-1]+pd.DateOffset(months=1), periods=n_forecast, freq='MS')
fig.add_trace(go.Scatter(x=future_dates, y=forecast, mode='lines+markers', name='Pronóstico futuro', line=dict(dash='dash')))
fig.update_layout(title='Pronóstico de pasajeros con Holt-Winters', xaxis_title='Fecha', yaxis_title='Pasajeros')
fig.show()

# Calcular el RMSE de cada método (usando solo el ajuste, no el forecast futuro)
# Para cada método, alinear los índices antes de calcular el error
def rmse_alineado(real, pred):
    mask = pred.notna()
    return np.sqrt(mean_squared_error(real[mask], pred[mask]))

rmse_sma = rmse_alineado(df['passengers'], df['SMA_12'])
rmse_ses = rmse_alineado(df['passengers'], df['SES'])
rmse_brown = rmse_alineado(df['passengers'], df['Brown'])
rmse_holt = rmse_alineado(df['passengers'], df['Holt'])
rmse_hw = rmse_alineado(df['passengers'], df['HW'])

print("RMSE Promedio Móvil 12 meses:", round(rmse_sma,2))
print("RMSE Suavización Exponencial Simple:", round(rmse_ses,2))
print("RMSE Brown:", round(rmse_brown,2))
print("RMSE Holt:", round(rmse_holt,2))
print("RMSE Holt-Winters:", round(rmse_hw,2))
# Elegir el mejor método
errores = {
    'Promedio Móvil 12 meses': rmse_sma,
    'Suavización Exponencial Simple': rmse_ses,
    'Brown': rmse_brown,
    'Holt': rmse_holt,
    'Holt-Winters': rmse_hw
}
mejor_metodo = min(errores, key=errores.get)
print(f"\nEl mejor método según el RMSE es: {mejor_metodo}")

# Explicación sencilla
print("\nEl RMSE (Root Mean Squared Error) mide el error promedio entre los valores reales y los pronosticados. El método con menor RMSE es el que mejor se ajusta a los datos históricos. En este caso, el mejor método es el que tiene el RMSE más bajo.")

RMSE Promedio Móvil 12 meses: 48.01
RMSE Suavización Exponencial Simple: 47.65
RMSE Brown: 33.52
RMSE Holt: 33.71
RMSE Holt-Winters: 12.24

El mejor método según el RMSE es: Holt-Winters

El RMSE (Root Mean Squared Error) mide el error promedio entre los valores reales y los pronosticados. El método con menor RMSE es el que mejor se ajusta a los datos históricos. En este caso, el mejor método es el que tiene el RMSE más bajo.


## 12. Aplicación del algoritmo en el contexto de analítica y big data

En entornos de **big data** y analítica avanzada, los métodos de forecasting se escalan usando herramientas como:

- **Apache Spark (PySpark):** Permite procesar grandes volúmenes de datos y aplicar modelos de series de tiempo distribuidos.
- **Prophet:** Escalable y fácil de usar para datos con estacionalidad múltiple.
- **Plataformas cloud:** Azure ML, AWS Forecast, Google Cloud AI ofrecen soluciones listas para producción.

Estas herramientas permiten automatizar y escalar el pronóstico en empresas con grandes volúmenes de datos y necesidades de análisis en tiempo real.

## 13. Ejemplos de Pronósticos con otros conjuntos de datos

En esta sección, aplicaremos diferentes métodos de pronóstico a series de tiempo ficticias de ventas, cada uno adaptado a las características específicas de los datos.

### 13.1 Ejemplo con serie económica sin tendencia/estacionalidad (SES)

El **método de suavización exponencial simple (SES)** es adecuado para series que no presentan tendencia ni estacionalidad. Este método asigna un peso mayor a las observaciones más recientes, lo que permite capturar cambios en el nivel de la serie.

#### Cargar datos
Usaremos un conjunto de datos ficticio que representa ventas mensuales sin tendencia ni estacionalidad.

In [12]:
# Crear un DataFrame de ejemplo
np.random.seed(0)
meses = pd.date_range(start='2020-01-01', periods=24, freq='M')
ventas = np.random.normal(loc=200, scale=10, size=len(meses))
df_ses = pd.DataFrame({'Fecha': meses, 'Ventas': ventas})
df_ses.set_index('Fecha', inplace=True)

# Aplicar SES
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
ses_model = SimpleExpSmoothing(df_ses['Ventas']).fit(smoothing_level=0.2, optimized=False)
df_ses['SES'] = ses_model.fittedvalues

# Visualizar resultados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ses.index, y=df_ses['Ventas'], mode='lines', name='Ventas reales'))
fig.add_trace(go.Scatter(x=df_ses.index, y=df_ses['SES'], mode='lines', name='Suavización Exponencial Simple'))
fig.update_layout(title='Suavización Exponencial Simple', xaxis_title='Fecha', yaxis_title='Ventas')
fig.show()

In [13]:
resultado = adfuller(df_ses['Ventas'])
print(f'ADF Statistic: {resultado[0]}')
print(f'p-value: {resultado[1]}')

if resultado[1] <= 0.05:
    print("La serie es estacionaria")
else:
    print("La serie no es estacionaria")

ADF Statistic: -4.093664084563589
p-value: 0.0009921454460620955
La serie es estacionaria


In [14]:
# Calcular el RMSE entre las ventas reales y los valores suavizados
rmse = sqrt(mean_squared_error(df_ses['Ventas'], df_ses['SES']))
print(f'RMSE del modelo SES: {rmse:.2f}')

RMSE del modelo SES: 10.72


In [15]:
# Pronóstico de los próximos 1 períodos
n_forecast = 1
forecast = ses_model.forecast(n_forecast)

# Crear fechas futuras
fechas_futuras = pd.date_range(start=df_ses.index[-1] + pd.DateOffset(months=1), periods=n_forecast, freq='M')

# Mostrar resultados
df_forecast = pd.DataFrame({'Fecha': fechas_futuras, 'Pronóstico SES': forecast})
print(df_forecast)

                Fecha  Pronóstico SES
2022-01-31 2022-01-31      199.010935


### 13.2 Ejemplo con serie de ventas con tendencia (Brown/Holt)

El **método de Brown** o **Holt** se utiliza para series que presentan una tendencia lineal. Este método extiende la suavización exponencial simple al incluir un componente de tendencia.

#### Cargar datos
Usaremos un conjunto de datos ficticio que representa ventas mensuales con una tendencia creciente.


In [16]:
# Crear un DataFrame de ejemplo con tendencia
np.random.seed(1)
tendencia = np.linspace(200, 300, 24) + np.random.normal(loc=0, scale=5, size=24)
df_holt = pd.DataFrame({'Fecha': meses, 'Ventas': tendencia})
df_holt.set_index('Fecha', inplace=True)

# Aplicar Holt
from statsmodels.tsa.holtwinters import Holt
holt_model = Holt(df_holt['Ventas']).fit()
df_holt['Holt'] = holt_model.fittedvalues

# Visualizar resultados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_holt.index, y=df_holt['Ventas'], mode='lines', name='Ventas reales'))
fig.add_trace(go.Scatter(x=df_holt.index, y=df_holt['Holt'], mode='lines', name='Método de Holt'))
fig.update_layout(title='Método de Holt', xaxis_title='Fecha', yaxis_title='Ventas')
fig.show()

In [17]:
# Crear variable de tiempo como índice numérico
tiempo = np.arange(len(df_holt))
slope, intercept, r_value, p_value, std_err = linregress(tiempo, df_holt['Ventas'])

print(f'Pendiente estimada: {slope:.2f}')
print(f'p-value: {p_value:.4f}')
print(f'R²: {r_value**2:.4f}')

if p_value < 0.05:
    print("Hay evidencia estadística de tendencia.")
else:
    print("No hay evidencia clara de tendencia.")

Pendiente estimada: 4.42
p-value: 0.0000
R²: 0.9700
Hay evidencia estadística de tendencia.


In [18]:
# Crear gráfico de barras para los valores de autocorrelación
acf_vals = acf(df_holt['Ventas'], nlags=23)
lags = list(range(len(acf_vals)))

fig = go.Figure()
fig.add_trace(go.Bar(x=lags, y=acf_vals, name='Autocorrelación'))

# Agregar línea de referencia en y=0
fig.add_shape(type='line', x0=0, x1=max(lags), y0=0, y1=0,
              line=dict(color='gray', dash='dash'))

fig.update_layout(
    title='Autocorrelación de la serie (ACF) - Holt',
    xaxis_title='Lag',
    yaxis_title='Autocorrelación',
    yaxis=dict(range=[-1, 1])
)

fig.show()

In [19]:
result = seasonal_decompose(df_holt['Ventas'], model='additive', period=12)

# Medir la varianza explicada por la estacionalidad
var_total = np.var(df_holt['Ventas'])
var_estacional = np.var(result.seasonal)
porcentaje_estacional = (var_estacional / var_total) * 100

print(f'Varianza explicada por la estacionalidad: {porcentaje_estacional:.2f}%')

Varianza explicada por la estacionalidad: 2.99%


In [20]:
rmse_holt = sqrt(mean_squared_error(df_holt['Ventas'], df_holt['Holt']))
print(f'RMSE del modelo Holt: {rmse_holt:.2f}')

RMSE del modelo Holt: 8.34


In [21]:
# Número de períodos a pronosticar
n_periodos = 3
forecast_holt = holt_model.forecast(n_periodos)

# Crear fechas futuras
fechas_futuras = pd.date_range(start=df_holt.index[-1] + pd.DateOffset(months=1), periods=n_periodos, freq='M')

# Mostrar resultados
df_forecast_holt = pd.DataFrame({'Fecha': fechas_futuras, 'Pronóstico Holt': forecast_holt})
print(df_forecast_holt)

                Fecha  Pronóstico Holt
2022-01-31 2022-01-31       309.429623
2022-02-28 2022-02-28       314.761966
2022-03-31 2022-03-31       320.094309


### 13.3 Ejemplo con serie de ventas con tendencia y estacionalidad (Holt-Winters)

El **método de Holt-Winters** es adecuado para series que presentan tanto tendencia como estacionalidad. Este método incluye un componente estacional adicional que permite capturar patrones periódicos.

#### Cargar datos
Usaremos un conjunto de datos ficticio que representa ventas mensuales con una tendencia creciente y estacionalidad.


In [22]:
# Crear un DataFrame de ejemplo con tendencia y estacionalidad
np.random.seed(2)
estacionalidad = 40 * np.sin(np.linspace(0, 3 * np.pi, 24))
ventas_hw = tendencia + estacionalidad + np.random.normal(loc=0, scale=5, size=24)
df_hw = pd.DataFrame({'Fecha': meses, 'Ventas': ventas_hw})
df_hw.set_index('Fecha', inplace=True)

# Aplicar Holt-Winters
from statsmodels.tsa.holtwinters import ExponentialSmoothing
hw_model = ExponentialSmoothing(df_hw['Ventas'], trend='add', seasonal='add', seasonal_periods=12).fit()
df_hw['HW'] = hw_model.fittedvalues

# Visualizar resultados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_hw.index, y=df_hw['Ventas'], mode='lines', name='Ventas reales'))
fig.add_trace(go.Scatter(x=df_hw.index, y=df_hw['HW'], mode='lines', name='Holt-Winters'))
fig.update_layout(title='Método Holt-Winters', xaxis_title='Fecha', yaxis_title='Ventas')
fig.show()

In [23]:
# Calcular autocorrelaciones
acf_vals_hw = acf(df_hw['Ventas'], nlags=23)
lags_hw = list(range(len(acf_vals_hw)))

# Crear gráfico de barras
fig = go.Figure()
fig.add_trace(go.Bar(x=lags_hw, y=acf_vals_hw, name='Autocorrelación'))

# Línea de referencia en y=0
fig.add_shape(type='line', x0=0, x1=max(lags_hw), y0=0, y1=0,
              line=dict(color='gray', dash='dash'))

fig.update_layout(
    title='Autocorrelación para detectar estacionalidad (Holt-Winters)',
    xaxis_title='Lag',
    yaxis_title='Autocorrelación',
    yaxis=dict(range=[-1, 1])
)

fig.show()

In [24]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df_hw['Ventas'], model='additive', period=12)

# Medir la varianza explicada por la estacionalidad
var_total = np.var(df_hw['Ventas'])
var_estacional = np.var(result.seasonal)
porcentaje_estacional = (var_estacional / var_total) * 100

print(f'Varianza explicada por la estacionalidad: {porcentaje_estacional:.2f}%')

Varianza explicada por la estacionalidad: 21.65%


In [25]:
rmse_hw = sqrt(mean_squared_error(df_hw['Ventas'], df_hw['HW']))
print(f'RMSE del modelo Holt-Winters: {rmse_hw:.2f}')

RMSE del modelo Holt-Winters: 11.67


In [26]:
# Número de períodos a pronosticar
n_periodos = 3
forecast_hw = hw_model.forecast(n_periodos)

# Crear fechas futuras
fechas_futuras = pd.date_range(start=df_hw.index[-1] + pd.DateOffset(months=1), periods=n_periodos, freq='M')

# Mostrar resultados
df_forecast_hw = pd.DataFrame({'Fecha': fechas_futuras, 'Pronóstico HW': forecast_hw})
print(df_forecast_hw)

                Fecha  Pronóstico HW
2022-01-31 2022-01-31     306.651331
2022-02-28 2022-02-28     311.159629
2022-03-31 2022-03-31     324.903329


## Conclusión

A continuación, se presenta un resumen de cuándo utilizar cada modelo de pronóstico según las características de la serie de tiempo:

| Modelo                     | Características de la serie                       | Uso recomendado                          |
|----------------------------|--------------------------------------------------|-----------------------------------------|
| Suavización Exponencial Simple (SES) | Sin tendencia ni estacionalidad                  | Series estables sin patrones claros    |
| Método de Brown/Holt      | Con tendencia lineal, sin estacionalidad         | Series con crecimiento o decrecimiento  |
| Holt-Winters              | Con tendencia y estacionalidad                   | Series con patrones estacionales claros |

Este resumen proporciona una guía rápida para seleccionar el modelo de pronóstico más adecuado según las características de los datos.